In [2]:
import pandas as pd
import numpy as np
import re

#for sentiment analyzing
import nltk
import random
from nltk.tokenize import word_tokenize

#for basic preprocessing
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.text import Text
import string, re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/siddhipatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siddhipatel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/siddhipatel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [45]:
#HELPER FUNCTIONS

#remove punctuations, numbers, and special characters
def remove_special(df):
    df['Text'] = df['Text'].str.replace("[^\w\d'\s]+", '')
    return df

In [ ]:
def clean_up(text):

    

In [46]:
biden_url = 'https://raw.githubusercontent.com/srpatel2000/Elections2020/master/Tweets/2020_Elections/Biden_Tweets.csv'
biden_df = pd.read_csv(biden_url, error_bad_lines= False)
biden_df

,timestamp,Username,Retweets,Text,Mentions,Hashtags
0,Thu Apr 25 16:59:59 +0000 2019,980275870655893504,0,"And, we have President Trump to thank.",NaN,NaN
1,Thu Apr 25 16:59:59 +0000 2019,102468365,0,pic.twitter.com/PxypFuZzdb,NaN,NaN
2,Thu Apr 25 16:59:58 +0000 2019,827325184432603136,0,The “hang em high” references were less than s...,NaN,NaN
3,Thu Apr 25 16:59:57 +0000 2019,753734119893393408,0,"Donald Trump Makes Joe Biden’s Point, Mocking ...",NaN,NaN
4,Thu Apr 25 16:59:57 +0000 2019,140074335,0,"Factually speaking, that’s not accurate. Trump...",NaN,NaN
...,...,...,...,...,...,...
52795,Sat Apr 11 11:49:37 +0000 2020,1236376132825612290,0,Yes he have to for some whom haven’t hear it b...,NaN,NaN
52796,Sat Apr 11 11:49:36 +0000 2020,874403447667404800,0,"No, no... this way Joe, this way........... th...",NaN,NaN
52797,Sat Apr 11 11:49:36 +0000 2020,1010964754708303872,0,You want more of this? #TaraReadeBlackoutpic.t...,NaN,#TaraReadeBlackoutpic
52798,Sat Apr 11 11:49:36 +0000 2020,3775550189,0,LA voter- it literally doesn’t matter. You can...,NaN,NaN


In [47]:
biden_df_test = remove_special(biden_df)
biden_df_test

,timestamp,Username,Retweets,Text,Mentions,Hashtags
0,Thu Apr 25 16:59:59 +0000 2019,980275870655893504,0,And we have President Trump to thank,NaN,NaN
1,Thu Apr 25 16:59:59 +0000 2019,102468365,0,pictwittercomPxypFuZzdb,NaN,NaN
2,Thu Apr 25 16:59:58 +0000 2019,827325184432603136,0,The hang em high references were less than subtle,NaN,NaN
3,Thu Apr 25 16:59:57 +0000 2019,753734119893393408,0,Donald Trump Makes Joe Bidens Point Mocking Ca...,NaN,NaN
4,Thu Apr 25 16:59:57 +0000 2019,140074335,0,Factually speaking thats not accurate Trump ha...,NaN,NaN
...,...,...,...,...,...,...
52795,Sat Apr 11 11:49:37 +0000 2020,1236376132825612290,0,Yes he have to for some whom havent hear it be...,NaN,NaN
52796,Sat Apr 11 11:49:36 +0000 2020,874403447667404800,0,No no this way Joe this way that's the kitchen,NaN,NaN
52797,Sat Apr 11 11:49:36 +0000 2020,1010964754708303872,0,You want more of this TaraReadeBlackoutpictwit...,NaN,#TaraReadeBlackoutpic
52798,Sat Apr 11 11:49:36 +0000 2020,3775550189,0,LA voter it literally doesnt matter You can st...,NaN,NaN


In [44]:
string = "warhol's art used many types of media, including hand drawing, painting, photography, silk screening, sculpture, film, music."
print (re.sub("[^\w\d'\s]+",'',string))



warhol's art used many types of media including hand drawing painting photography silk screening sculpture film music


In [54]:
pd.read_csv('https://raw.githubusercontent.com/srpatel2000/Elections2020/master/president_polls%20(1).csv')

,question_id,poll_id,cycle,state,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,...,ranked_choice_reallocated,created_at,notes,url,stage,race_id,answer,candidate_name,candidate_party,pct
0,121384,65688,2020,NaN,744,Ipsos,71,Reuters,Ipsos,154.0,...,False,4/8/20 09:36,NaN,https://www.reuters.com/article/us-health-coro...,general,NaN,Biden,Joseph R. Biden Jr.,DEM,43.0
1,121384,65688,2020,NaN,744,Ipsos,71,Reuters,Ipsos,154.0,...,False,4/8/20 09:36,NaN,https://www.reuters.com/article/us-health-coro...,general,NaN,Trump,Donald Trump,REP,37.0
2,121427,65698,2020,NaN,568,YouGov,1213,Yahoo News,YouGov,391.0,...,False,4/9/20 13:19,NaN,https://docs.cdn.yougov.com/1ayt0i64g6/2020040...,general,NaN,Biden,Joseph R. Biden Jr.,DEM,49.0
3,121427,65698,2020,NaN,568,YouGov,1213,Yahoo News,YouGov,391.0,...,False,4/9/20 13:19,NaN,https://docs.cdn.yougov.com/1ayt0i64g6/2020040...,general,NaN,Trump,Donald Trump,REP,40.0
4,121428,65698,2020,NaN,568,YouGov,1213,Yahoo News,YouGov,391.0,...,False,4/9/20 13:20,NaN,https://docs.cdn.yougov.com/1ayt0i64g6/2020040...,general,NaN,Sanders,Bernard Sanders,DEM,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4142,92081,57026,2020,Ohio,383,Public Policy Polling,918,DEM,Public Policy Polling,263.0,...,False,12/6/18 14:45,NaN,https://www.cleveland.com/politics/2018/11/thi...,general,6246.0,Trump,Donald Trump,REP,46.0
4143,92082,57026,2020,Ohio,383,Public Policy Polling,918,DEM,Public Policy Polling,263.0,...,False,12/6/18 14:46,NaN,https://www.cleveland.com/politics/2018/11/thi...,general,6246.0,Warren,Elizabeth Warren,DEM,43.0
4144,92082,57026,2020,Ohio,383,Public Policy Polling,918,DEM,Public Policy Polling,263.0,...,False,12/6/18 14:46,NaN,https://www.cleveland.com/politics/2018/11/thi...,general,6246.0,Trump,Donald Trump,REP,49.0
4145,92078,57025,2020,NaN,399,Rasmussen Reports/Pulse Opinion Research,NaN,NaN,Rasmussen Reports/Pulse Opinion Research,277.0,...,False,12/6/18 14:40,NaN,http://www.rasmussenreports.com/public_content...,general,6210.0,Obama,Michelle Obama,DEM,50.0


In [55]:
pd.read_csv('https://raw.githubusercontent.com/srpatel2000/Elections2020/master/president_general_polls_2016.csv')

,cycle,branch,type,matchup,forecastdate,state,startdate,enddate,pollster,grade,...,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin,multiversions,url,poll_id,question_id,createddate,timestamp
0,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/3/2016,11/6/2016,ABC News/Washington Post,A+,...,45.20163,41.72430,4.626221,NaN,NaN,https://www.washingtonpost.com/news/the-fix/wp...,48630,76192,11/7/16,09:35:33 8 Nov 2016
1,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/1/2016,11/7/2016,Google Consumer Surveys,B,...,43.34557,41.21439,5.175792,NaN,NaN,https://datastudio.google.com/u/0/#/org//repor...,48847,76443,11/7/16,09:35:33 8 Nov 2016
2,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/2/2016,11/6/2016,Ipsos,A-,...,42.02638,38.81620,6.844734,NaN,NaN,http://projects.fivethirtyeight.com/polls/2016...,48922,76636,11/8/16,09:35:33 8 Nov 2016
3,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/4/2016,11/7/2016,YouGov,B,...,45.65676,40.92004,6.069454,NaN,NaN,https://d25d2506sfb94s.cloudfront.net/cumulus_...,48687,76262,11/7/16,09:35:33 8 Nov 2016
4,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/3/2016,11/6/2016,Gravis Marketing,B-,...,46.84089,42.33184,3.726098,NaN,NaN,http://www.gravispolls.com/2016/11/final-natio...,48848,76444,11/7/16,09:35:33 8 Nov 2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12619,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/8/16,New Hampshire,7/9/2016,7/18/2016,University of New Hampshire,B+,...,40.24983,43.04717,6.924110,NaN,NaN,https://cola.unh.edu/sites/cola.unh.edu/files/...,44650,68189,7/21/16,09:14:14 8 Nov 2016
12620,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/8/16,Wisconsin,10/21/2016,11/2/2016,Ipsos,A-,...,46.54218,38.96884,NaN,NaN,NaN,http://www.reuters.com/statesofthenation/,48259,75560,11/3/16,09:14:14 8 Nov 2016
12621,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/8/16,New York,8/7/2016,8/10/2016,Siena College,A,...,53.83622,32.47939,3.881193,NaN,NaN,https://www.siena.edu/assets/files/news/SNY081...,44852,68743,8/15/16,09:14:14 8 Nov 2016
12622,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/8/16,Virginia,9/30/2016,10/6/2016,Ipsos,A-,...,49.57558,39.96954,NaN,NaN,NaN,http://www.reuters.com/statesofthenation/,46675,72969,10/10/16,09:14:14 8 Nov 2016
